INSPER - Big Data e Computação em Nuvem

# Big Data e Computação em Nuvem - Projeto Final: Análise de Atraso de Voos

**Integrantes**:
- Flávio
- Hélio
- Michel

In [9]:
#Importando as bibliotecas necessárias
import seaborn as sns
import pyspark.sql.functions as f
from pyspark.sql.types import StringType
from matplotlib import pyplot as plt
from pyspark.sql.functions import col, when
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [1]:
# Criar a sessao do Spark

spark = SparkSession \
            .builder \
            .master("local[4]") \
            .appName("nyc_<mudar-nome>") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,com.microsoft.azure:azure-storage:8.6.6") \
            .getOrCreate()

In [2]:
STORAGE_ACCOUNT = 'dlspadseastusprod'
CONTAINER = 'big-data-comp-nuvem'
FOLDER = 'airline-delay'
TOKEN = 'lSuH4ZI9BhOFEhCF/7ZQbrpPBIhgtLcPDfXjJ8lMxQZjaADW4p6tcmiZGDX9u05o7FqSE2t9d2RD+ASt0YFG8g=='

spark.conf.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)

In [3]:
df = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/*".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, inferSchema=True)

- Definir o escopo do negócio: qùal será a definição do atraso?
- A análise toda é do ponto do vista da empresa, consumidor ou governo (regulador)?
- Criar variavel resposta
- Tratamento de missing values
- Verificar a qualidade das informações (estao no tipo certo?)
- Criar colunas novas? Vamos manter todas as colunas para a etapa de modelagem?
- Vamos usar outras variaveis (preço de petroleo, pois QAV representa 41% do custo em 2022, segundo a ANAC)? Como era o desempenho economico em cada ano?
- Faremos filtros para retirar anos outliers?
- Vamos retirar outliers? Verificar anos atípicos
- verificar a influência de variáveis externa (desempenho economico, preço do petróleo, tempo)
- Matriz de correlação
- Analise bivariada: % atrasos por aeroporto, por regiao


In [4]:
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: double (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: d

In [14]:
df.limit(5).toPandas()

/opt/conda/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,1204,DCA,EWR,1100.0,1058.0,-2.0,18.0,1116.0,...,62.0,68.0,42.0,199.0,NaN,NaN,NaN,NaN,NaN,None
1,2009-01-01,XE,1206,EWR,IAD,1510.0,1509.0,-1.0,28.0,1537.0,...,82.0,75.0,43.0,213.0,NaN,NaN,NaN,NaN,NaN,None
2,2009-01-01,XE,1207,EWR,DCA,1100.0,1059.0,-1.0,20.0,1119.0,...,70.0,62.0,36.0,199.0,NaN,NaN,NaN,NaN,NaN,None
3,2009-01-01,XE,1208,DCA,EWR,1240.0,1249.0,9.0,10.0,1259.0,...,77.0,56.0,37.0,199.0,NaN,NaN,NaN,NaN,NaN,None
4,2009-01-01,XE,1209,IAD,EWR,1715.0,1705.0,-10.0,24.0,1729.0,...,105.0,77.0,40.0,213.0,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
#Remover a coluna Unnamed

In [12]:
# numero de linhas
total_rows = df.count()

In [13]:
total_rows

61556964

In [15]:
#Verificando os missing values
df.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).toPandas()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,0,0,0,0,0,1,935723,940675,963901,963896,...,60,1118754,1118753,0,50166224,50166224,50166224,50166224,50166224,61556964


In [ ]:
df.summary()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


In [ ]:
#tratar os missing values


In [ ]:
#Criar a variável reposta

In [ ]:
#Verificar a assimetria das variáveis?

In [ ]:
# Verificar se há valores negativos

### Criar novas colunas

A partir do histograma acima, vamos definir o seguinte critério para classificar as distâncias:

- próximo: abaixo de 500 milhas
- médio: entre 500 e 1000 milhas
- distante: acima de 1000 milhas

In [ ]:
#Criar coluna "Tipo de Distância" com o critério acima
df = df.withColumn("DISTANCE_TYPE", 
                   when(df['DISTANCE'] > 1000, "distant")
                   .when(df['DISTANCE'] > 500, "medium")
                   .otherwise("near"))

In [ ]:
#Remover colunas?

In [ ]:
#Verificar outliers
# Calcule a média e o desvio padrão
mean_value = df.agg(mean("value").alias("mean")).collect()[0]["mean"]
stddev_value = df.agg(stddev("value").alias("stddev")).collect()[0]["stddev"]

# Defina um limite para identificar outliers (por exemplo, 3 desvios padrão da média)
lower_limit = mean_value - 3 * stddev_value
upper_limit = mean_value + 3 * stddev_value

# Identifique outliers
outliers = df.filter((col("value") < lower_limit) | (col("value") > upper_limit))

# Contagem de outliers
num_outliers = outliers.count()

# Mostra o resultado
print(f"Número de outliers: {num_outliers}")
outliers.show()

In [ ]:
# Combine as features em um vetor usando VectorAssembler
vector_assembler = VectorAssembler(inputCols=df.columns[1:], outputCol="features")
df = vector_assembler.transform(df)

# Calcule a matriz de correlação
correlation_matrix = df.select("features").stat.corr("features")

# Mostra o resultado
print("Matriz de Correlação:")
correlation_matrix.show()

In [ ]:
#Analise bivariada: % atrasos por aeroporto, por regiao